# Static Graph Rendering with plot_static()

This notebook demonstrates how to render graphs as static images (SVG, PNG) for documentation, reports, presentations, and CI pipelines.

**Key insight**: Layout (where nodes go) and rendering (output format) are separate concerns:
- Use `plot()` for interactive Graphistry GPU visualization
- Use `plot_static()` for static image export

`plot_static()` works with **any layout** - UMAP, ring, graphviz, manual positions, or let graphviz compute one.

## Setup

Requires `pygraphviz` for rendering. Install with:
```bash
# System: apt-get install graphviz graphviz-dev
# Python: pip install pygraphviz
# Or: pip install graphistry[pygraphviz]
```

In [ ]:
import pandas as pd
import graphistry

# Check if pygraphviz is available
try:
    import pygraphviz
    HAS_PYGRAPHVIZ = True
    print('pygraphviz available')
except ImportError:
    HAS_PYGRAPHVIZ = False
    print('pygraphviz not installed - code examples shown but not executed')

In [ ]:
# Sample graph data
edges_df = pd.DataFrame({
    'src': ['a', 'a', 'b', 'c', 'd'],
    'dst': ['b', 'c', 'c', 'd', 'e']
})

nodes_df = pd.DataFrame({
    'id': ['a', 'b', 'c', 'd', 'e'],
    'type': ['start', 'middle', 'middle', 'middle', 'end'],
    'label': ['Start', 'Step 1', 'Step 2', 'Step 3', 'End']
})

g = graphistry.edges(edges_df, 'src', 'dst').nodes(nodes_df, 'id')
print(f'Graph: {len(nodes_df)} nodes, {len(edges_df)} edges')

## Quick Start

Basic `plot_static()` renders to SVG and auto-displays in Jupyter:

In [ ]:
if HAS_PYGRAPHVIZ:
    # Auto-displays inline, returns SVG bytes
    svg_bytes = g.plot_static()
    print(f'Returned {len(svg_bytes)} bytes of SVG')
else:
    print('# Example: g.plot_static()')

## Output Formats

Several output engines are available:

In [ ]:
if HAS_PYGRAPHVIZ:
    # SVG (default)
    svg = g.plot_static(format='svg')
    print(f'SVG: {len(svg)} bytes')
    
    # PNG
    png = g.plot_static(format='png')
    print(f'PNG: {len(png)} bytes')
    
    # DOT source text
    dot = g.plot_static(engine='graphviz-dot')
    print(f'DOT: {len(dot)} chars')
    print(dot[:200] + '...')
else:
    print('''# Examples:
svg = g.plot_static(format='svg')      # SVG bytes
png = g.plot_static(format='png')      # PNG bytes  
dot = g.plot_static(engine='graphviz-dot')  # DOT text
mmd = g.plot_static(engine='mermaid-code')  # Mermaid text''')

In [ ]:
if HAS_PYGRAPHVIZ:
    # Mermaid DSL (for GitHub markdown, etc.)
    mermaid = g.plot_static(engine='mermaid-code')
    print('Mermaid output:')
    print(mermaid)
else:
    print('# Example: g.plot_static(engine="mermaid-code")')

## Styling Options - Common Cases

`plot_static()` accepts `graph_attr`, `node_attr`, and `edge_attr` dictionaries for styling.

### Graph-level styling (graph_attr)

| Goal | Code |
|------|------|
| Horizontal layout | `graph_attr={'rankdir': 'LR'}` |
| Vertical (default) | `graph_attr={'rankdir': 'TB'}` |
| White background | `graph_attr={'bgcolor': 'white'}` |
| Compact spacing | `graph_attr={'nodesep': '0.3', 'ranksep': '0.3'}` |

In [ ]:
if HAS_PYGRAPHVIZ:
    # Horizontal layout with white background
    g.plot_static(
        graph_attr={'rankdir': 'LR', 'bgcolor': 'white'}
    )
else:
    print("# g.plot_static(graph_attr={'rankdir': 'LR', 'bgcolor': 'white'})")

### Node-level styling (node_attr)

| Goal | Code |
|------|------|
| Filled boxes | `node_attr={'shape': 'box', 'style': 'filled'}` |
| Rounded boxes | `node_attr={'shape': 'box', 'style': 'rounded,filled'}` |
| Color all nodes | `node_attr={'fillcolor': 'lightblue'}` |
| Font styling | `node_attr={'fontname': 'Helvetica', 'fontsize': '12'}` |

In [ ]:
if HAS_PYGRAPHVIZ:
    # Rounded filled boxes
    g.plot_static(
        graph_attr={'rankdir': 'LR'},
        node_attr={'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'}
    )
else:
    print("# g.plot_static(node_attr={'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'})")

### Edge-level styling (edge_attr)

| Goal | Code |
|------|------|
| Dashed edges | `edge_attr={'style': 'dashed'}` |
| Colored edges | `edge_attr={'color': 'gray'}` |
| Arrow style | `edge_attr={'arrowhead': 'vee'}` |

In [ ]:
if HAS_PYGRAPHVIZ:
    # Combined styling
    g.plot_static(
        prog='dot',
        graph_attr={'rankdir': 'LR', 'bgcolor': 'white'},
        node_attr={'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'},
        edge_attr={'color': 'gray', 'arrowhead': 'vee'}
    )
else:
    print('''# Combined styling example:
g.plot_static(
    prog='dot',
    graph_attr={'rankdir': 'LR', 'bgcolor': 'white'},
    node_attr={'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'},
    edge_attr={'color': 'gray', 'arrowhead': 'vee'}
)''')

### Per-node/per-edge styling (data-driven)

Add graphviz attribute columns to your dataframe for data-driven styling:

In [ ]:
# Add fillcolor column based on node type
nodes_styled = nodes_df.copy()
nodes_styled['fillcolor'] = nodes_styled['type'].map({
    'start': 'lightgreen',
    'middle': 'lightblue', 
    'end': 'lightyellow'
})
nodes_styled['shape'] = 'box'

g_styled = graphistry.edges(edges_df, 'src', 'dst').nodes(nodes_styled, 'id')

if HAS_PYGRAPHVIZ:
    # fillcolor and shape read from node columns
    g_styled.plot_static(
        graph_attr={'rankdir': 'LR'},
        node_attr={'style': 'filled'}  # enables fillcolor
    )
else:
    print('# Nodes with fillcolor column get per-node colors')
    print(nodes_styled[['id', 'type', 'fillcolor', 'shape']])

## Finding More Configuration Options

### Discover available attributes in PyGraphistry

In [ ]:
from graphistry.plugins_types.graphviz_types import (
    GRAPH_ATTRS,  # Valid graph-level attributes
    NODE_ATTRS,   # Valid node-level attributes  
    EDGE_ATTRS,   # Valid edge-level attributes
    PROGS,        # Layout programs
)

print('Layout programs (prog):', PROGS[:6], '...')
print(f'\nGraph attributes ({len(GRAPH_ATTRS)} total):', GRAPH_ATTRS[:10], '...')
print(f'\nNode attributes ({len(NODE_ATTRS)} total):', NODE_ATTRS[:10], '...')
print(f'\nEdge attributes ({len(EDGE_ATTRS)} total):', EDGE_ATTRS[:10], '...')

### External References

For the full list of attributes and values, see the Graphviz documentation:

- [Graphviz Attributes](https://graphviz.org/doc/info/attrs.html) - Complete attribute reference
- [Graphviz Shapes](https://graphviz.org/doc/info/shapes.html) - Node shapes (box, ellipse, diamond, etc.)
- [Graphviz Colors](https://graphviz.org/doc/info/colors.html) - Color names and schemes
- [Graphviz Arrows](https://graphviz.org/doc/info/arrows.html) - Arrow styles

## Layout Programs

When `plot_static()` computes layout (no existing x/y), the `prog` parameter controls the algorithm:

| Program | Best For |
|---------|----------|
| `dot` (default) | Hierarchies, DAGs, flowcharts |
| `neato` | Small undirected graphs |
| `fdp` | Larger undirected graphs |
| `sfdp` | Very large graphs (1000+ nodes) |
| `circo` | Circular/ring layouts |
| `twopi` | Radial layouts from root |

For a deep dive on layout algorithms, see the [graphviz.ipynb](graphviz.ipynb) notebook.

In [ ]:
if HAS_PYGRAPHVIZ:
    # Circular layout
    print('circo layout:')
    g.plot_static(prog='circo', node_attr={'style': 'filled', 'fillcolor': 'lightblue'})
else:
    print("# g.plot_static(prog='circo')")

## Works with Any Layout Source

`plot_static()` can render graphs that already have positions from other layout methods.

When `reuse_layout=True` (default) and x/y positions exist, graphviz preserves them.

In [ ]:
# Manual positions
nodes_with_pos = nodes_df.copy()
nodes_with_pos['x'] = [0, 100, 100, 200, 300]
nodes_with_pos['y'] = [50, 0, 100, 50, 50]

g_manual = graphistry.edges(edges_df, 'src', 'dst').nodes(nodes_with_pos, 'id')

if HAS_PYGRAPHVIZ:
    # Uses manual x/y positions
    g_manual.plot_static(
        reuse_layout=True,
        node_attr={'style': 'filled', 'fillcolor': 'lightyellow'}
    )
else:
    print('# g_manual.plot_static(reuse_layout=True)')
    print('# Preserves x/y columns from nodes dataframe')

### With UMAP layout

In [ ]:
# Example pattern (requires umap-learn):
print('''# UMAP layout -> static render
g2 = g.umap(X=['feature1', 'feature2'])
g2.plot_static(reuse_layout=True)  # Uses UMAP positions''')

### With ring layout

In [ ]:
# Example pattern:
print('''# Ring layout -> static render  
g2 = g.time_ring_layout('timestamp')
g2.plot_static(reuse_layout=True)  # Uses ring positions''')

### Force fresh layout

Set `reuse_layout=False` to ignore existing positions and let graphviz compute a new layout:

In [ ]:
if HAS_PYGRAPHVIZ:
    # Ignore manual positions, compute fresh dot layout
    g_manual.plot_static(
        reuse_layout=False,
        prog='dot',
        graph_attr={'rankdir': 'LR'},
        node_attr={'style': 'filled', 'fillcolor': 'lightpink'}
    )
else:
    print('# g_manual.plot_static(reuse_layout=False, prog="dot")')

## Saving Output

Use the `path` parameter to save to file (also returns bytes/text):

In [ ]:
import tempfile
import os

if HAS_PYGRAPHVIZ:
    with tempfile.TemporaryDirectory() as tmpdir:
        # Save SVG
        svg_path = os.path.join(tmpdir, 'graph.svg')
        g.plot_static(format='svg', path=svg_path)
        print(f'Saved SVG: {os.path.getsize(svg_path)} bytes')
        
        # Save PNG
        png_path = os.path.join(tmpdir, 'graph.png')
        g.plot_static(format='png', path=png_path)
        print(f'Saved PNG: {os.path.getsize(png_path)} bytes')
        
        # Save DOT source
        dot_path = os.path.join(tmpdir, 'graph.dot')
        g.plot_static(engine='graphviz-dot', path=dot_path)
        print(f'Saved DOT: {os.path.getsize(dot_path)} bytes')
else:
    print('''# Save to file:
g.plot_static(format='svg', path='graph.svg')
g.plot_static(format='png', path='graph.png')
g.plot_static(engine='graphviz-dot', path='graph.dot')''')

## CI/Documentation Pipelines

Generate diagrams at build time with safety caps:

In [ ]:
# Example CI pipeline pattern
print('''# Generate diagrams for docs
graphs = {
    'overview': g_overview,
    'detail': g_detail,
}

for name, graph in graphs.items():
    graph.plot_static(
        format='svg',
        path=f'docs/images/{name}.svg',
        max_nodes=200,   # Safety cap
        max_edges=400,
        graph_attr={'bgcolor': 'white'},
        node_attr={'style': 'filled'}
    )''')

## Summary

| Goal | Code |
|------|------|
| Quick SVG | `g.plot_static()` |
| PNG output | `g.plot_static(format='png')` |
| Save to file | `g.plot_static(path='graph.svg')` |
| Horizontal layout | `g.plot_static(graph_attr={'rankdir': 'LR'})` |
| Styled nodes | `g.plot_static(node_attr={'style': 'filled', 'fillcolor': 'lightblue'})` |
| Different layout | `g.plot_static(prog='circo')` |
| Preserve positions | `g.plot_static(reuse_layout=True)` |
| DOT source | `g.plot_static(engine='graphviz-dot')` |
| Mermaid | `g.plot_static(engine='mermaid-code')` |

### See Also

- [graphviz.ipynb](graphviz.ipynb) - Graphviz layout algorithms for interactive Graphistry visualization
- [Graphviz documentation](https://graphviz.org/documentation/) - Full attribute reference